# 演習III 第４回 ツイッターテキスト解析

出典：https://qiita.com/e10persona/items/7a7643b266c2bdfbf7d0

Google Colab 用

In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

必要なライブラリのインストール

In [ ]:
!pip install tweepy mecab-python3 wordcloud oseti

証明書関係エラー防止

In [ ]:
import os
import warnings
os.environ['CURL_CA_BUNDLE'] = ''
warnings.simplefilter('ignore')

ライブラリのインポートを行います。

In [ ]:
import tweepy
import MeCab
import csv
import json
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import warnings
import numpy as np
import pandas as pd
import oseti
import collections

## MeCab のテスト

※ MeCab は別途インストールしておくこと

In [ ]:
mecab = MeCab.Tagger("-Ochasen") 
malist = mecab.parse("すもももももももものうち")
print(malist)

## Tweet の取得

In [ ]:
import config # config.py
CK = config.CONSUMER_KEY
CS = config.CONSUMER_SECRET
AT = config.ACCESS_TOKEN
AS = config.ACCESS_TOKEN_SECRET
BT = config.BEARER_TOKEN

In [ ]:
client = tweepy.Client(bearer_token=BT)

## 特定ユーザーのツイートの取得

In [ ]:
tweets = client.get_users_tweets(id=id , tweet_fields=['context_annotations', 'created_at'], max_results=10)

In [ ]:
#'ndanyusi: 防衛大学校入試広報（公式）
user = client.get_user(username='ndanyusi')
id = user.data.id

tweets = client.get_users_tweets(id=id , tweet_fields=['context_annotations', 'created_at'], max_results=100)
for tweet in tweets.data:
    print(user.data.name)
    print(tweet.created_at)
    print(tweet.text)
    print("=================================")

### キーワードを含むツイートの取得

In [ ]:
# Replace with your own search query
query = '防衛大 -is:retweet'

results = []
# Replace the limit=10 with the maximum number of Tweets you want
for tweet in tweepy.Paginator(client.search_recent_tweets, query=query,
                              tweet_fields=['context_annotations', 'created_at'], user_fields=['username'],expansions=['author_id'], max_results=100).flatten(limit=100):
    username = client.get_user(id=tweet.data['author_id']).data.name
    dat = [username, tweet.created_at, tweet.text]
    print(dat)
    results.append(dat)
    print("=================================")

## 結果の保存

In [ ]:
df = pd.DataFrame(results, columns=['ユーザ名', '日時', 'テキスト'])
df.to_csv('tweets.csv', index=False, encoding='shift-jis', errors='ignore')
df

## 結果の読み込み

In [ ]:
df2 = pd.read_csv('tweets.csv', encoding='cp932')
df2

## MeCab で分析

### 分かち書き、品詞の抜き出し

In [ ]:
words = []
for tweet in df['テキスト']:
    node = mecab.parseToNode(tweet)
    while node:
        word = node.surface
        word_type = node.feature.split(",")[0]
 
        # "名詞", "動詞", "形容詞", "副詞"の中で選択したものを抽出
        if word_type in ["名詞", "動詞", "形容詞"]:
            words.append(word)
        node = node.next
    

### WordCloud として表示

In [ ]:
#wordcloudで出力するフォントを指定
font_path = "/System/Library/Fonts/ヒラギノ角ゴシック W4.ttc"
txt = " ".join(words)

# ストップワードの設定(特定のワードは入らないように設定)
stop_words = ['https', 't', 'co',  'RT']

#解析した単語、ストップワードを設定、背景の色は白
wordcloud = WordCloud(background_color="white", font_path=font_path, stopwords=set(stop_words),
    width=800,height=600).generate(txt)

fig = plt.figure(figsize=(30, 10))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

## ポジ・ネガ分析

In [ ]:
analyzer = oseti.Analyzer()
print(analyzer.analyze_detail("最後まで希望を捨てちゃいかん。あきらめたら、そこで試合終了だよ。"))
print(analyzer.analyze_detail("認めたくないものだな。自分自身の、若さゆえの過ちというものを。"))

In [ ]:
ave_senti = []
for tweet in df['テキスト']:
    print(tweet)
    senti = analyzer.analyze(tweet)
    print(analyzer.analyze_detail(tweet), np.mean(senti))
    ave_senti.append(np.mean(senti))
ave_senti

In [ ]:
hist, bins = np.histogram(ave_senti, bins=4)
plt.rcParams['font.family'] = "AppleGothic"
plt.pie(hist, labels=['ネガ', 'ややネガ', 'ややポジ', 'ポジ'], counterclock=False, startangle=90)

## 感情分析（Transformer 版）

In [ ]:
!pip install transformers fugashi ipadic

In [ ]:
from transformers import pipeline 
from transformers import AutoModelForSequenceClassification 
from transformers import BertJapaneseTokenizer 

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('daigo/bert-base-japanese-sentiment') 
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking') 
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer) 

In [ ]:
print(nlp("最後まで希望を捨てちゃいかん。あきらめたら、そこで試合終了だよ。"))
print(nlp("認めたくないものだな。自分自身の、若さゆえの過ちというものを。"))

In [ ]:
sentiments = []
for tweet in df['テキスト']:
    print(tweet)
    senti = nlp(tweet)
    print(senti)
    sentiments.append(senti[0]['score'])
print(sentiments)
print(np.mean(sentiments))

In [ ]:
hist, bins = np.histogram(sentiments, bins=4)
plt.rcParams['font.family'] = "AppleGothic"
plt.pie(hist, labels=['ネガ', 'ややネガ', 'ややポジ', 'ポジ'], counterclock=False, startangle=90)

## 課題
各自のテーマで Tweet を分析せよ。